In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt



# Importing files

In [2]:
db = pd.read_csv("OMNICM20651+Alternative+proteins+data+file+v2+public+use.csv")

In [3]:
db.head()

,DMQ_RESP_GENDER,DMQ_RESP_AGE_RECODED,DMQ_REGION,DMQ_HHCMP10,DMQ_KIDS02,DMQ_UK02EDU,DMQ_UK01MAR,DMQ_UK02INC,DMQ_UK02ETH,DMQ_EMP01,...,Q2073788224__14,Q2073788224__15,Q2073788224__16,Q2053017755,Q2079418049Wholeedibleinsects,Q2079418049Groundintoafoodforaddedproteinegbreadburgersfalaf,Q2079418049Madeintosweetsorjellies,Q2079418049Madeintobeveragesegsportsdrinksproteinshakes,Q2079418049Amealorproteinreplacement,DPWeight
0,2,2,5,3,1,5,3,4,2,6,...,0,0,1,,3,3,3,3,3,1.44043
1,2,1,10,3,0,5,1,3,2,2,...,,,,,1,2,2,2,3,0.66943
2,2,2,9,3,2,6,3,5,2,1,...,,,,,2,6,6,5,5,0.82482
3,1,3,4,6,4,2,3,5,2,1,...,0,0,0,,2,1,2,1,1,0.83600
4,2,3,2,6,3,3,3,4,2,6,...,,,,,3,3,3,3,3,0.98356


In [4]:
db.shape

(1930, 167)

In [5]:
db.dtypes

DMQ_RESP_GENDER                                                   int64
DMQ_RESP_AGE_RECODED                                              int64
DMQ_REGION                                                        int64
DMQ_HHCMP10                                                       int64
DMQ_KIDS02                                                        int64
                                                                 ...   
Q2079418049Groundintoafoodforaddedproteinegbreadburgersfalaf      int64
Q2079418049Madeintosweetsorjellies                                int64
Q2079418049Madeintobeveragesegsportsdrinksproteinshakes           int64
Q2079418049Amealorproteinreplacement                              int64
DPWeight                                                        float64
Length: 167, dtype: object

The only numeric variable is 'DMQ_HHCMP10' (Household size) and DP weight (The rest will need to be converted)

In [6]:
#converting the type of the variables
db = db.astype(object)
db['DPWeight'] = db['DPWeight'].astype(float)
db['DMQ_HHCMP10'] = db['DMQ_HHCMP10'].astype(int)

In [7]:
db.dtypes

DMQ_RESP_GENDER                                                  object
DMQ_RESP_AGE_RECODED                                             object
DMQ_REGION                                                       object
DMQ_HHCMP10                                                       int32
DMQ_KIDS02                                                       object
                                                                 ...   
Q2079418049Groundintoafoodforaddedproteinegbreadburgersfalaf     object
Q2079418049Madeintosweetsorjellies                               object
Q2079418049Madeintobeveragesegsportsdrinksproteinshakes          object
Q2079418049Amealorproteinreplacement                             object
DPWeight                                                        float64
Length: 167, dtype: object

In [8]:
db.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1930 entries, 0 to 1929
Columns: 167 entries, DMQ_RESP_GENDER to DPWeight
dtypes: float64(1), int32(1), object(165)
memory usage: 2.5+ MB


# Looking empty values

In [9]:
#The dataset has many empty values that were inputed with a space that Pandas does not recognize. To go around that we will use
#the following code

db = db.replace(' ',np.nan)

C:\Temp\ipykernel_17656\3257374438.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  db = db.replace(' ',np.nan)


In [10]:
pd.set_option("display.max_rows" , None)

db.isna().sum()

DMQ_RESP_GENDER                                                    0
DMQ_RESP_AGE_RECODED                                               0
DMQ_REGION                                                         0
DMQ_HHCMP10                                                        0
DMQ_KIDS02                                                         0
DMQ_UK02EDU                                                        0
DMQ_UK01MAR                                                        0
DMQ_UK02INC                                                        0
DMQ_UK02ETH                                                       22
DMQ_EMP01                                                          0
DMQ_EMP01_RECODED                                                  0
UK01SG                                                             0
DMQ_UK01SG_RECODED                                                 0
Q2089090438Edibleinsects                                           0
Q2089090438Labgrownmeatsometimesre

In [11]:
percent_missing = db.isnull().sum() * 100 / len(db)
percent_missing

DMQ_RESP_GENDER                                                   0.000000
DMQ_RESP_AGE_RECODED                                              0.000000
DMQ_REGION                                                        0.000000
DMQ_HHCMP10                                                       0.000000
DMQ_KIDS02                                                        0.000000
DMQ_UK02EDU                                                       0.000000
DMQ_UK01MAR                                                       0.000000
DMQ_UK02INC                                                       0.000000
DMQ_UK02ETH                                                       1.139896
DMQ_EMP01                                                         0.000000
DMQ_EMP01_RECODED                                                 0.000000
UK01SG                                                            0.000000
DMQ_UK01SG_RECODED                                                0.000000
Q2089090438Edibleinsects 

#In the survey, looking at the data dictionary, we can see that the many empty values are related to multi-choice questions, 
and empty values means the person did not reply to this question specifically. I will add the value 2 to this columns that means (no answers). 

For DMQ_UK02ETH (it's the etnicity, so I implied that the person did not want to reply, in this case we will replace the empty value for 24 (Prefer not to answer)

For Q2055782931 the question is about how safe the person thinks is to eat edible insects, to avoid loosing important data by deleting these rows and as the percentage of empty values is low we will replace it by the mode

For Q2053017755 it has 100% missing valus so I will drop the column.

In [12]:
db['DMQ_UK02ETH'] = db['DMQ_UK02ETH'].replace(np.nan, '24')
db['Q2055782931'] = db['Q2055782931'].replace(np.nan, db['Q2055782931'].mode()[0])

In [13]:
percent_missing = db.isnull().sum() * 100 / len(db)
percent_missing

DMQ_RESP_GENDER                                                   0.000000
DMQ_RESP_AGE_RECODED                                              0.000000
DMQ_REGION                                                        0.000000
DMQ_HHCMP10                                                       0.000000
DMQ_KIDS02                                                        0.000000
DMQ_UK02EDU                                                       0.000000
DMQ_UK01MAR                                                       0.000000
DMQ_UK02INC                                                       0.000000
DMQ_UK02ETH                                                       0.000000
DMQ_EMP01                                                         0.000000
DMQ_EMP01_RECODED                                                 0.000000
UK01SG                                                            0.000000
DMQ_UK01SG_RECODED                                                0.000000
Q2089090438Edibleinsects 

In [14]:
db = db.drop('Q2053017755', axis=1)

In [15]:
db = db.replace(np.nan, '2')

In [16]:
percent_missing = db.isnull().sum() * 100 / len(db)
percent_missing

DMQ_RESP_GENDER                                                 0.0
DMQ_RESP_AGE_RECODED                                            0.0
DMQ_REGION                                                      0.0
DMQ_HHCMP10                                                     0.0
DMQ_KIDS02                                                      0.0
DMQ_UK02EDU                                                     0.0
DMQ_UK01MAR                                                     0.0
DMQ_UK02INC                                                     0.0
DMQ_UK02ETH                                                     0.0
DMQ_EMP01                                                       0.0
DMQ_EMP01_RECODED                                               0.0
UK01SG                                                          0.0
DMQ_UK01SG_RECODED                                              0.0
Q2089090438Edibleinsects                                        0.0
Q2089090438Labgrownmeatsometimesreferredtoascult

In [17]:
# Checking for duplicates 
db = db.drop_duplicates()

# Exploratory Analysis

In [18]:
db["DMQ_RESP_GENDER"].value_counts()

DMQ_RESP_GENDER
2    1064
1     866
Name: count, dtype: int64

In [19]:
db.dtypes

DMQ_RESP_GENDER                                                  object
DMQ_RESP_AGE_RECODED                                             object
DMQ_REGION                                                       object
DMQ_HHCMP10                                                       int32
DMQ_KIDS02                                                       object
DMQ_UK02EDU                                                      object
DMQ_UK01MAR                                                      object
DMQ_UK02INC                                                      object
DMQ_UK02ETH                                                      object
DMQ_EMP01                                                        object
DMQ_EMP01_RECODED                                                object
UK01SG                                                           object
DMQ_UK01SG_RECODED                                               object
Q2089090438Edibleinsects                                        

In [20]:
#For the graphs need to change the data to text since it was encoded in the original database (info available in the dictionary)
db_1 = db
db_1["DMQ_RESP_GENDER"] = db_1["DMQ_RESP_GENDER"].replace(1, 'Male')
db_1["DMQ_RESP_GENDER"] = db_1["DMQ_RESP_GENDER"].replace(2, 'Female')

db_1["DMQ_RESP_AGE_RECODED"] = db_1["DMQ_RESP_AGE_RECODED"].replace(1, '16-24')
db_1["DMQ_RESP_AGE_RECODED"] = db_1["DMQ_RESP_AGE_RECODED"].replace(2, '25-34')
db_1["DMQ_RESP_AGE_RECODED"] = db_1["DMQ_RESP_AGE_RECODED"].replace(3, '35-44')
db_1["DMQ_RESP_AGE_RECODED"] = db_1["DMQ_RESP_AGE_RECODED"].replace(4, '45-54')
db_1["DMQ_RESP_AGE_RECODED"] = db_1["DMQ_RESP_AGE_RECODED"].replace(5, '55-64')
db_1["DMQ_RESP_AGE_RECODED"] = db_1["DMQ_RESP_AGE_RECODED"].replace(6, '65-75')

db_1["DMQ_REGION"] = db_1["DMQ_REGION"].replace(1, 'North East')
db_1["DMQ_REGION"] = db_1["DMQ_REGION"].replace(2, 'North West')
db_1["DMQ_REGION"] = db_1["DMQ_REGION"].replace(3, 'Yorkshire and The Humber')
db_1["DMQ_REGION"] = db_1["DMQ_REGION"].replace(4, 'West Midlands')
db_1["DMQ_REGION"] = db_1["DMQ_REGION"].replace(5, 'East Midlands')
db_1["DMQ_REGION"] = db_1["DMQ_REGION"].replace(6, 'East of England')
db_1["DMQ_REGION"] = db_1["DMQ_REGION"].replace(7, 'South West')
db_1["DMQ_REGION"] = db_1["DMQ_REGION"].replace(8, 'South East')
db_1["DMQ_REGION"] = db_1["DMQ_REGION"].replace(9, 'Greater London')
db_1["DMQ_REGION"] = db_1["DMQ_REGION"].replace(10, 'Wales')
db_1["DMQ_REGION"] = db_1["DMQ_REGION"].replace(11, 'Scotland')
db_1["DMQ_REGION"] = db_1["DMQ_REGION"].replace(12, 'Northern Ireland')

db_1["DMQ_UK02EDU"] = db_1["DMQ_UK02EDU"].replace(1, 'Primary school')
db_1["DMQ_UK02EDU"] = db_1["DMQ_UK02EDU"].replace(2, 'Secondary school (age under 15 years old)')
db_1["DMQ_UK02EDU"] = db_1["DMQ_UK02EDU"].replace(3, 'GNVQ / GSVQ / GCSE/ SCE standard')
db_1["DMQ_UK02EDU"] = db_1["DMQ_UK02EDU"].replace(4, 'NVQ1, NVQ2')
db_1["DMQ_UK02EDU"] = db_1["DMQ_UK02EDU"].replace(5, 'NVQ3/ SCE Higher Grade/ Advanced GNVQ/ GCE A/AS or similar')
db_1["DMQ_UK02EDU"] = db_1["DMQ_UK02EDU"].replace(6, 'NVQ4 / HNC / HND / Bachelor\'s degree or similar')
db_1["DMQ_UK02EDU"] = db_1["DMQ_UK02EDU"].replace(7, 'NVQ5 or post-graduate diploma')

db_1["DMQ_UK01MAR"] = db_1["DMQ_UK01MAR"].replace(1, 'Never Married (Single)')
db_1["DMQ_UK01MAR"] = db_1["DMQ_UK01MAR"].replace(2, 'Domestic Partner (Living as a couple)')
db_1["DMQ_UK01MAR"] = db_1["DMQ_UK01MAR"].replace(3, 'Married / Civil partnership')
db_1["DMQ_UK01MAR"] = db_1["DMQ_UK01MAR"].replace(4, 'Separated')
db_1["DMQ_UK01MAR"] = db_1["DMQ_UK01MAR"].replace(5, 'Divorced')
db_1["DMQ_UK01MAR"] = db_1["DMQ_UK01MAR"].replace(6, 'Widowed')

db_1["DMQ_UK02INC"] = db_1["DMQ_UK02INC"].replace(1, 'Under £5,000')
db_1["DMQ_UK02INC"] = db_1["DMQ_UK02INC"].replace(2, '£5,000 - 9,999')
db_1["DMQ_UK02INC"] = db_1["DMQ_UK02INC"].replace(3, '£10,000 - 14,999')
db_1["DMQ_UK02INC"] = db_1["DMQ_UK02INC"].replace(4, '£15,000 - 19,999')
db_1["DMQ_UK02INC"] = db_1["DMQ_UK02INC"].replace(5, '£20,000 - 24,999')
db_1["DMQ_UK02INC"] = db_1["DMQ_UK02INC"].replace(6, '£25,000 - 34,999')
db_1["DMQ_UK02INC"] = db_1["DMQ_UK02INC"].replace(7, '£35,000 - 44,999')
db_1["DMQ_UK02INC"] = db_1["DMQ_UK02INC"].replace(8, '£45,000 - 54,999')
db_1["DMQ_UK02INC"] = db_1["DMQ_UK02INC"].replace(9, '£55,000 - 99,999')
db_1["DMQ_UK02INC"] = db_1["DMQ_UK02INC"].replace(10, '£100,000 or more')
db_1["DMQ_UK02INC"] = db_1["DMQ_UK02INC"].replace(11, 'Prefer not to answer')

db_1["DMQ_UK02ETH"] = db_1["DMQ_UK02ETH"].replace('1', 'White')
db_1["DMQ_UK02ETH"] = db_1["DMQ_UK02ETH"].replace('2', 'English / Welsh / Scottish / Northern Irish / British')
db_1["DMQ_UK02ETH"] = db_1["DMQ_UK02ETH"].replace('3', 'Irish')
db_1["DMQ_UK02ETH"] = db_1["DMQ_UK02ETH"].replace('4', 'Gypsy or Irish Traveller')
db_1["DMQ_UK02ETH"] = db_1["DMQ_UK02ETH"].replace('5', 'Any other White background')
db_1["DMQ_UK02ETH"] = db_1["DMQ_UK02ETH"].replace('6', 'Mixed / multiple ethnic groups')
db_1["DMQ_UK02ETH"] = db_1["DMQ_UK02ETH"].replace('7', 'White and Black Caribbean')
db_1["DMQ_UK02ETH"] = db_1["DMQ_UK02ETH"].replace('8', 'White and Black African')
db_1["DMQ_UK02ETH"] = db_1["DMQ_UK02ETH"].replace('9', 'White and Asian')
db_1["DMQ_UK02ETH"] = db_1["DMQ_UK02ETH"].replace('10', 'Any other Mixed / multiple ethnic background')
db_1["DMQ_UK02ETH"] = db_1["DMQ_UK02ETH"].replace('11', 'Asian / Asian British')
db_1["DMQ_UK02ETH"] = db_1["DMQ_UK02ETH"].replace('12', 'Indian')
db_1["DMQ_UK02ETH"] = db_1["DMQ_UK02ETH"].replace('13', 'Pakistani')
db_1["DMQ_UK02ETH"] = db_1["DMQ_UK02ETH"].replace('14', 'Bangladeshi')
db_1["DMQ_UK02ETH"] = db_1["DMQ_UK02ETH"].replace('15', 'Chinese')
db_1["DMQ_UK02ETH"] = db_1["DMQ_UK02ETH"].replace('16', 'Any other Asian background')
db_1["DMQ_UK02ETH"] = db_1["DMQ_UK02ETH"].replace('17', 'Black / African / Caribbean / Black British')
db_1["DMQ_UK02ETH"] = db_1["DMQ_UK02ETH"].replace('18', 'African')
db_1["DMQ_UK02ETH"] = db_1["DMQ_UK02ETH"].replace('19', 'Caribbean')
db_1["DMQ_UK02ETH"] = db_1["DMQ_UK02ETH"].replace('20', 'Any other Black / African / Caribbean background')
db_1["DMQ_UK02ETH"] = db_1["DMQ_UK02ETH"].replace('21', 'Other ethnic group')
db_1["DMQ_UK02ETH"] = db_1["DMQ_UK02ETH"].replace('22', 'Arab')
db_1["DMQ_UK02ETH"] = db_1["DMQ_UK02ETH"].replace('23', 'Any other ethnic group')
db_1["DMQ_UK02ETH"] = db_1["DMQ_UK02ETH"].replace('24', 'Prefer not to answer')

db_1["UK01SG"] = db_1["UK01SG"].replace('1', 'A - Upper middle class')
db_1["UK01SG"] = db_1["UK01SG"].replace('2', 'B - Middle class')
db_1["UK01SG"] = db_1["UK01SG"].replace('3', 'C1 - Lower middle class')
db_1["UK01SG"] = db_1["UK01SG"].replace('4', 'C2 - Skilled working class')
db_1["UK01SG"] = db_1["UK01SG"].replace('5', 'D - Working class')
db_1["UK01SG"] = db_1["UK01SG"].replace('6', 'E - Lower level of subsistence')

db_1["Q2089090438Edibleinsects"] = db_1["Q2089090438Edibleinsects"].replace(1, 'Yes, and I know what it means')
db_1["Q2089090438Edibleinsects"] = db_1["Q2089090438Edibleinsects"].replace(2, 'Yes, but I don’t know what it means')
db_1["Q2089090438Edibleinsects"] = db_1["Q2089090438Edibleinsects"].replace(3, 'No')
db_1["Q2089090438Edibleinsects"] = db_1["Q2089090438Edibleinsects"].replace(4, 'Don t know')


db_1["Q2089090438Edibleinsects"]

0                                Don t know
1       Yes, but I don’t know what it means
2             Yes, and I know what it means
3             Yes, and I know what it means
4                                Don t know
5       Yes, but I don’t know what it means
6             Yes, and I know what it means
7             Yes, and I know what it means
8                                        No
9             Yes, and I know what it means
10            Yes, and I know what it means
11            Yes, and I know what it means
12            Yes, and I know what it means
13                                       No
14      Yes, but I don’t know what it means
15            Yes, and I know what it means
16      Yes, but I don’t know what it means
17            Yes, and I know what it means
18            Yes, and I know what it means
19            Yes, and I know what it means
20      Yes, but I don’t know what it means
21            Yes, and I know what it means
22            Yes, and I know wh

## How many people have heard of Edible insects by gender

In [33]:
data = db_1.groupby(['Q2089090438Edibleinsects', 'DMQ_RESP_GENDER']).count()

In [38]:
labels = data.index
male = data['Male']
Female = data['Female']

KeyError: 'Male'

In [37]:
# plot data in grouped manner of bar type 
plt.bar(data['DMQ_RESP_AGE_RECODED']-0.2, data['Q2089090438Edibleinsects'], width) 
plt.bar(data['DMQ_RESP_AGE_RECODED']+0.2, data['DMQ_RESP_GENDER'], width) 

KeyError: 'Q2089090438Edibleinsects'